### **PHISHING VECTOR GENERATOR** 🐟

In [138]:
import pandas as pd
import numpy as np
import json
from copy import deepcopy
import os
import time
import requests
import re
from user_browsing import user_browsing
from xml.etree import ElementTree as ET
from urllib.parse import urlparse
from os import path

In [139]:
class PHISH_FVG:

    def __init__(self, url, fichero = 'html_dump'):

        self.url = url
        self.base, self.path = self.process_url(self.url)
        self.fv = np.array([ -1 for i in range(19)])
        self.html = ''
        self.fichero = fichero

        self.user = user_browsing()
        self.user.set_standard_header(self.base)


    def process_url(self, url):

        parsed = urlparse(url)
        base = parsed.netloc
        return base, base + '/'.join(path.split('/')[:-1])


    def get_source_code(self):

        response = requests.get(self.url) #, headers=self.user.header) #proxies=user.proxies, cookies=user.cookies)

        if response.status_code != 400:
            with open(self.fichero, 'wb') as f:
                f.write(response.content)
                f.close()

        self.html = response.content.decode("utf-8")

    
    def url_validator(self, url):

        parsed = urlparse(url)
        return parsed.netloc != ''


    def set_f9(self):

        form_pattern = '(?:<form)(.*?)(?:<\\/form>)'
        form_found = re.findall(form_pattern, self.html)

        if len(form_found) > 0:

            action_pattern = '(?:action=\")(.*?)(?:\")'
            action_content = re.findall(action_pattern, form_found[0])[0]
            print(action_content)


            if (len(action_content) <=1) or (len(re.findall(r'javascript:void\(0\)', action_content)) > 0):
                self.fv[8] = 1

            elif len(re.findall(r'^(.*\.php)$', action_content)) > 0 and self.base != self.process_url(action_content)[0]:
                self.fv[8] = 1

            elif action_content[0] != '/' and self.base != self.process_url(action_content)[0]:
                self.fv[8] = 1
            
            else:
                self.fv[8] = 0
                
        else:
            self.fv[8] = 0

        


In [142]:
ph_entity = PHISH_FVG('https://fdeageadfahgeafeahg.azurewebsites.net/renner/inicio/login.php')
ph_entity.get_source_code()
ph_entity.set_f9()

['https://ubuvirtual.ubu.es/holsdasdasda.php']